# Environment

## Libraries

In [26]:
import os, pathlib, sys, re, apache_beam as beam
from apache_beam.io import ReadFromText
from apache_beam.io import WriteToText
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.options.pipeline_options import SetupOptions

## Working Directory

In [3]:
workingdir=os.getcwd()
d=[d for d in os.listdir(workingdir)]
n=0
while not set(['notebook']).issubset(set(d)):
    workingdir=str(pathlib.Path(workingdir).parents[0])

    d=[d for d in os.listdir(str(workingdir))]
    n+=1
    if n>5:
        break
sys.path.insert(0, workingdir)
os.chdir(workingdir)

## Apache Beam and GCP Settings

In [10]:
options = PipelineOptions()
GoogleCloudOptions = beam.options.pipeline_options.GoogleCloudOptions
StandardOptions = beam.options.pipeline_options.StandardOptions
google_cloud_options = options.view_as(GoogleCloudOptions)
google_cloud_options.project = 'axa-ch-machine-learning-dev'
google_cloud_options.job_name = 'nlp_text_classification_preprocessing'
google_cloud_options.staging_location = 'gs://nlp_text_classification'
google_cloud_options.temp_location = 'gs://nlp_text_classification'
options.view_as(StandardOptions).runner = 'DataflowRunner'

# Pipeline

In [22]:
#p = beam.Pipeline(options=options)
p = beam.Pipeline(argv=sys.argv)

In [23]:
#input_document = 'gs://dataflow-samples/shakespeare/kinglear.txt'
input_document = 'C:/Users/C311723/OneDrive - AXA/Desktop/dickens.txt'

In [24]:
(p
    | beam.io.ReadFromText(input_document) # read input (PCollection object representing lines)
    | 'ExtractWords' >> beam.FlatMap(lambda x: re.findall(r'[A-Za-z\']+', x)) # extract words (output: PCollection)
    | beam.combiners.Count.PerElement() #calculate word occurences (output: key/value pairs of words and counts)
    | beam.MapTuple(lambda word, count: '%s: %s' % (word, count)) # transform k/v pair to writeable string
    | beam.io.WriteToText('C:/Users/C311723/OneDrive - AXA/Desktop/dickens_analysis.txt') # write output
)

<PCollection[WriteToText/Write/WriteImpl/FinalizeWrite.None] at 0x2539e0dc588>

In [25]:
result = p.run()